In [9]:
def parse_text(text):
    import MeCab
    import re
 
    mecab = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
    parsed = mecab.parse(text)
    lines = parsed.split('\n')
    lines = lines[0:-2]
    word_list = []
 
    for line in lines:
        tmp = re.split('\t|,', line)  # 正規表現\tか, でsplit
        
        # 名詞のみ対象
        if tmp[1] in ["名詞"]:
            word_list.append(tmp[7]) # 1: 表層形, 7: 原形
 
    return " " . join(word_list)

In [20]:
import pandas as pd 
import re
from wordcloud import WordCloud
 
FONT_PATH = "/System/Library/Fonts/ヒラギノ丸ゴ ProN W4.ttc"
myNAME = "成瀬 大悟"
NAME = "永野大河"

talks = pd.DataFrame(columns=["日付", "時刻", "発言者", "内容"])

f = open(f"./トーク/[LINE] {NAME}とのトーク.txt", encoding="UTF-8")
all_data = f.readlines()
f.close()

title = all_data[0]
save_date = all_data[1]
data = all_data[3:]

# 下の正規表現ほど制約が緩いので、順番を下にすること
p_date = re.compile(r"\d{4}\/\d{2}\/\d{2}\(.\)\n")
p_talk = re.compile(r"\d{2}:\d{2}\t[\w|\s]+\t(\w*\n*)")
p_cancel = re.compile(r".*メッセージの送信を取り消しました.*")
p_textonly = re.compile(r"(.+\n*)")
p_br  = re.compile(r"\n")

my_texts = ""
friend_texts = ""

for i, line in enumerate(data):
    if p_date.match(line):
        pass
    elif p_talk.match(line):
        tmp = line.split("\t")[1]
        if tmp == NAME:
            friend_texts += p_talk.match(line).groups()[0]
        elif tmp == myNAME:
            my_texts += p_talk.match(line).groups()[0]
        else:
            print("error")
            
    elif p_cancel.match(line):
        pass
    elif p_textonly.match(line):
        if tmp == NAME:
            friend_texts += p_textonly.match(line).groups()[0]
        elif tmp == myNAME:
            my_texts += p_textonly.match(line).groups()[0]
        else:
            print("error")
    elif p_br.match(line):
        pass
    else:
        print("\n\texception occurs!!")
        print(f"line :{i}")
        print(line)

my_words = parse_text(my_texts)
friend_words = parse_text(friend_texts)

stop_words =  open("MySlothLib.txt", encoding="utf8").readlines()
for i,  w in enumerate(stop_words):
    stop_words[i] = w.rstrip('\n')
mylist = ["笑", "笑笑", "通話", "時間", "今日", "明日", "ん", "の", "する", "ある", "やる", "いい", "こと", "そう", "それ", "おれ", "なん", "俺", "オレ", "これ", "http", "https"]
stop_words.extend(mylist)

my_wc = WordCloud(max_font_size=100, \
               background_color="white", \
               stopwords=set(stop_words), \
               width=400, height=400, \
               font_path=FONT_PATH).generate(my_words)
fr_wc = WordCloud(max_font_size=100, \
               background_color="white", \
               stopwords=set(stop_words), \
               width=400, height=400, \
               font_path=FONT_PATH).generate(friend_words)

my_wc.to_file(f"{myNAME}_to_{NAME}.png")
fr_wc.to_file(f"{NAME}_to_{myNAME}.png")